In [43]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
#import os
#import sys

#from selenium import webdriver

pd.set_option('display.max_columns', None)

In [95]:
#url = 'https://www.redfin.com/zipcode/94605'
url = 'https://www.redfin.com/zipcode/94605/filter/include=sold-3yr'
hdr = {'User-Agent': 'Mozilla/5.0'}

response = requests.get(url, headers=hdr)

assert response.status_code==200, "HTML code isn't 200."



In [96]:
page = response.text
len(page)

920975

In [97]:
soup = BeautifulSoup(page,"lxml")

In [87]:
soup.prettify()

'<!DOCTYPE html>\n<html lang="en">\n <head>\n  <!-- Server: phantom-3 -->\n  <!-- Time generated: Wed Apr 18 2018 12:43:18 GMT-0700 (PDT) -->\n  <script>\n   /*! LAB.js (LABjs :: Loading And Blocking JavaScript)\n    v2.0.3 (c) Kyle Simpson\n    MIT License\n*/\n!function(e){function t(e){return"[object Function]"==Object.prototype.toString.call(e)}function n(e){return"[object Array]"==Object.prototype.toString.call(e)}function r(e,t){var n=/^\\w+\\:\\/\\//;return/^\\/\\/\\/?/.test(e)?e=location.protocol+e:n.test(e)||"/"==e.charAt(0)||(e=(t||"")+e),n.test(e)?e:("/"==e.charAt(0)?v:m)+e}function i(e,t){for(var n in e)e.hasOwnProperty(n)&&(t[n]=e[n]);return t}function c(e){for(var t=!1,n=0;n<e.scripts.length;n++)e.scripts[n].ready&&e.scripts[n].exec_trigger&&(t=!0,e.scripts[n].exec_trigger(),e.scripts[n].exec_trigger=null);return t}function s(e,t,n,r){e.onload=e.onreadystatechange=function(){e.readyState&&"complete"!=e.readyState&&"loaded"!=e.readyState||t[n]||(e.onload=e.onreadystatechan

In [89]:
len(soup.prettify())

146338

In [91]:
tag = soup.title
type(tag)

bs4.element.Tag

In [107]:
#soup.find_all('a')
# soup.find_all("div", class_="HomeCardImage")
links = []
for link in soup.find_all("a", class_="cover-all"):
    links.append(link['href'])
    


In [109]:
print(links)
len(links)

['/CA/Oakland/2812-76th-Ave-94605/home/2036652', '/CA/Oakland/6129-Old-Quarry-Loop-94605/home/12115918', '/CA/Oakland/5921-Hilton-St-94605/home/568204', '/CA/Oakland/3055-Bantry-Ave-94605/home/1161573', '/CA/Oakland/4009-Malcolm-Ave-94605/home/1993133', '/CA/Oakland/8301-Aster-Ave-94605/home/1417868', '/CA/Oakland/8009-Earl-St-94605/home/579005', '/CA/Oakland/5528-Picardy-Dr-S-94605/home/567257', '/CA/Oakland/5514-Picardy-Dr-N-94605/home/1563765', '/CA/Oakland/7841-Surrey-Ln-94605/home/592451', '/CA/Oakland/4751-Dunkirk-Ave-94605/home/661538', '/CA/Oakland/3101-Partridge-Ave-94605/home/2040553', '/CA/Oakland/6144-Old-Quarry-Loop-94605/home/21670339', '/CA/Oakland/7560-Hansom-Dr-94605/home/843353', '/CA/Oakland/3240-62nd-Ave-94605/home/1925170', '/CA/Oakland/3680-Malcolm-Ave-94605/home/1930765', '/CA/Oakland/3200-Revere-Ave-94605/home/599097', '/CA/Oakland/60-Maggiora-Dr-94605/home/981030', '/CA/Oakland/2-Riviera-Ct-94605/home/1952844', '/CA/Oakland/5115-Fairhill-Ct-94605/home/1820927']

20

In [90]:
for link in soup.find_all(class='address'):
    print(link)
    print("\n")

SyntaxError: invalid syntax (<ipython-input-90-823779596bb5>, line 1)

In [58]:
type(soup.find_all("a")[0])

bs4.element.Tag

### Open downloaded CSV from Redfin to check its contents and use as a reference

In [37]:
!ls

LICENSE                        explore_luther.ipynb
README.md                      redfin_2018-04-18-08-51-54.csv


In [47]:
rf_csv = pd.read_csv('redfin_2018-04-18-08-51-54.csv')
rf_csv.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY', 'STATE',
       'ZIP', 'PRICE', 'BEDS', 'BATHS', 'LOCATION', 'SQUARE FEET', 'LOT SIZE',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'HOA/MONTH', 'STATUS',
       'NEXT OPEN HOUSE START TIME', 'NEXT OPEN HOUSE END TIME',
       'URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
       'SOURCE', 'MLS#', 'FAVORITE', 'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

## Learn BS4

In [74]:
soup2 = BeautifulSoup('<b class="boldest">Extremely bold</b>','lxml')
tag2 = soup2.b
type(tag)

bs4.element.Tag

In [75]:
tag2.name

'b'

In [76]:
tag2['class']

['boldest']

In [77]:
tag2.attrs

{'class': ['boldest']}

In [78]:
tag2.get('class')

['boldest']

In [79]:
tag2.string

'Extremely bold'